# PPO Mario # 

(inspired by youtube tutorial with adjustments, also barely used)

### Set-Up ###

Installs *

In [ ]:
#install the game and nes_py
%pip install gym_super_mario_bros nes_py

Imports *

In [ ]:
#Import Game
import gym_super_mario_bros

#Import Joypad (basically gives access to a virtual controller)
from nes_py.wrappers import JoypadSpace

#Use SIMPLE Controls (setting type of actions available)
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

Setup *

In [ ]:
#sets which environment (v0, v1... etc) is being used
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)

    (see num of actions)

In [ ]:
#number of possible action combinations
env.action_space

    (see actions possible)

In [ ]:
#list of possible actions (will change if you are not using simple movement)
SIMPLE_MOVEMENT

### Start The Game And Render Untrained Mario ###

Start the game and you will see Mario taking random movements until he dies. This is the computer playing mario without any training.

In [ ]:
#flag for restart
done = True
#loop through each frame of the game
for step in range(100000):
  if done:
    env.reset()
  #take a random action for the current "step" and recieve state, reward and done (dead of not)
  state, reward, done, info = env.step(env.action_space.sample())
  #show game on screen
  env.render()
#end/close game
env.close()

    (if on close the window stays running run the following line of code)

In [ ]:
env.close()

### Preprocessing Environment ###

Installs *

In [ ]:
#install pytorch first to run on GPU
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

#fixes issues with installing stable-baseline
!pip install setuptools==65.5.0

#install stable-baseline for algos... etc.
%pip install stable-baselines3[extra]

If stable baselines won't install try the following

In [ ]:
%pip install git+https://github.com/carlosluis/stable-baselines3@fix_tests

Imports *

In [ ]:
#Import wrapper for grayscaling
from gym.wrappers import GrayScaleObservation

#Import wrappers for vectorization
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv

#Import matplotlib to show the impact of frame stacking
from matplotlib import pyplot as plt

Preprocess *

In [ ]:
#create base environment
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)

#apply grayscale
env = GrayScaleObservation(env, keep_dim=True)

#wrap in dummy environment
env = DummyVecEnv([lambda: env])

#stack frames
env = VecFrameStack(env, 4, channels_order='last')

### Training ###

Imports *

In [ ]:
#os for file maintnance
import os

#PPO algo
from stable_baselines3 import PPO

#Base Callback for saving models
from stable_baselines3.common.callbacks import BaseCallback

  Model Saving Class *

In [ ]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

Model And Saving Setup *

In [ ]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#pathing for data logs
CHECKPOINT_DIR = '' #logged models and different training points
LOG_DIR = '' #tensor board for info on the model

#setup model saving callback
#check_freq = the amount of iterations between saves (watch storage)
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

#creates the neural network and the AI itself (CnnPolicy is best for image processing)
model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001, n_steps=512)

Training *

In [ ]:
#train the model so it can learn
#total_timesteps = how many itterations you want to trian for (should be put in the millions)
model.learn(total_timesteps=TIME_STEPS, callback=callback)

    (manual save model)

In [ ]:
#manual save of the model just run at any iteration
model.save('PPO_manual_save')

### Start The Game And Render Trained Mario ###

After a training is complete you can load in a model and run it to see how to agent has improved.

Select Model *

In [ ]:
#select model to run
model_path = 'train/...' #type model path as a string here

Load Model *

In [ ]:
#load model
model = PPO.load(model_path)

Run/Start Game *

In [ ]:
state = env.reset()
while True:
    action, _ = model.predict(state)
    state, reward, done, info = env.step(action)
    env.render()